In [ ]:
import os
import random
import trimesh
import numpy as np
import mujoco
import mujoco_viewer
import glob
import math
from lxml import etree

# The code models DP-Flex.xml as closed chained fingers

open = True # defines how to control the hand. 
#True if open-chained (no underactuation)
#False if closed-chained (keep underactuation)

# Function swaps attributes in the .xml using lxml etree
def swap_par(tree, element_type, element_name, attribute_name, new_value):   
    element = tree.find(f'.//{element_type}[@name="{element_name}"]')
    element.set(attribute_name, new_value)

f = "C:\\Users\\373529\\Desktop\\models_hub\\DP-FLEX_code\\DP-Flex_closed_kinematics.xml"

xml_path = "DP-Flex_closed_kinematics.xml"

if open == True:
    stiffness = 0 # reset the stiffness
else:
    stiffness = 0.1 # underactuated mechanism

# Rewriting the stiffness attribute
tree = etree.parse(xml_path)

swap_par(tree, 'joint', 'Joint_left_finray_proxy', 'stiffness', f"{stiffness}")
swap_par(tree, 'joint', 'Joint_right_finray_proxy', 'stiffness', f"{stiffness}")
swap_par(tree, 'joint', 'Joint_thumb_finray_proxy', 'stiffness', f"{stiffness}")

tree.write(xml_path, pretty_print=True, xml_declaration=True, encoding='UTF-8')
model = mujoco.MjModel.from_xml_path(f)
data = mujoco.MjData(model)

id_JLA = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_left_abduction_p")
id_JRA = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_right_abduction_p")
id_JTA = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_thumb_abduction_p")

id_JLF_o = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_left_flexion_p")
id_JRF_o = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_right_flexion_p")
id_JTF_o = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_thumb_flexion_p")

id_JLP = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_left_finray_proxy_p")
id_JRP = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_right_finray_proxy_p")
id_JTP = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_thumb_finray_proxy_p")

id_JTR = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_thumb_rotation_p")

id_JLF_c = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_left_dynamixel_crank_p")
id_JRF_c = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_right_dynamixel_crank_p")
id_JTF_c = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_ACTUATOR, "Joint_thumb_dynamixel_crank_p")

viewer = mujoco_viewer.MujocoViewer(model, 
                                    data, 
                                    title="HO", 
                                    width=2560, 
                                    height=1440)

while viewer.is_alive:
    
    # abductions control
    data.ctrl[id_JLA] = np.deg2rad(0) #0..90
    data.ctrl[id_JRA] = np.deg2rad(45) #0..90
    data.ctrl[id_JTA] = np.deg2rad(0) #0..90
       
    # thumb rotation control
    data.ctrl[id_JTR] = np.deg2rad(30) #-90..90
    
    if open == True: # we control proximal and distal phalanges (F_o and P respectively)
        data.ctrl[id_JLF_o] = np.deg2rad(45) #0..90
        data.ctrl[id_JRF_o] = np.deg2rad(90) #0..90
        data.ctrl[id_JTF_o] = np.deg2rad(0) #0..90
        
        data.ctrl[id_JLP] = np.deg2rad(45) #0..90
        data.ctrl[id_JRP] = np.deg2rad(90) #0..90
        data.ctrl[id_JTP] = np.deg2rad(0) #0..90
        
    else: # we control dynamixel crank (F_c) only
        data.ctrl[id_JLF_c] = np.deg2rad(45) #0..90
        data.ctrl[id_JRF_c] = np.deg2rad(90) #0..90
        data.ctrl[id_JTF_c] = np.deg2rad(0) #0..90
    
    mujoco.mj_step(model, data)

    viewer.render()

viewer.close()